In [1]:
from fairseq.models.transformer_lm import TransformerLanguageModel
from torch.utils.data import DataLoader
import torch 
import numpy as np
import os

os.environ["TRUST_REMOTE_CODE"] = "True"
local_rank = 0
device = "cuda:" + str(local_rank) if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(device)
device_type = "cuda" if device.startswith("cuda") else "cpu"

#Load Pretrained Model
fairseq_model = TransformerLanguageModel.from_pretrained('/base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/')

2024-08-15 21:20:59 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
/home/jovyan/fairseq/fairseq/tasks/multires_hubert_pretraining.py:154: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  dictionaries = [ (Dictionary.load(f"{label_dir}/dict.{label}.txt") if label is not "" else None ) for label in self.cfg.labels]
2024-08-15 21:21:00 | INFO | fairseq.file_utils | loading archive file /base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/
/home/jovyan/fairseq/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the fun

In [8]:
print(fairseq_model)

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerLanguageModel(
      (decoder): TransformerDecoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): AdaptiveInput(
          (embeddings): ModuleList(
            (0): Sequential(
              (0): Embedding(20000, 1024, padding_idx=1)
              (1): Linear(in_features=1024, out_features=1024, bias=False)
            )
            (1): Sequential(
              (0): Embedding(40000, 256)
              (1): Linear(in_features=256, out_features=1024, bias=False)
            )
            (2): Sequential(
              (0): Embedding(207744, 64)
              (1): Linear(in_features=64, out_features=1024, bias=False)
            )
          )
        )
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0-15): 16 x TransformerDecoderLayerBase(
            (dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dro

In [11]:
print(fairseq_model.models[0].decoder.layers[0].activation_dropout_module.p)

0.1


In [39]:
config = fairseq_model.cfg
for k in config.keys():
    print(k + 'arguments:')
    if config[k] is None:
        continue
    for l,v in config[k].items():
        print(l,v)
    print('-------------------------------')




_namearguments:
commonarguments:
_name None
no_progress_bar False
log_interval 25
log_format json
log_file None
aim_repo None
aim_run_hash None
tensorboard_logdir /checkpoint/myleott/tensorboard_logs/2020-05-23/adaptive_lm_wiki103.fp16.no_c10d.transformer_lm_wiki103.adaptive_loss.nag.clip0.1.warmup16000.initlr1e-07.sampletok3072.breaknone.maxtok3072.updatefreq3.seed2.ngpu8
wandb_project None
azureml_logging False
seed 2
cpu False
tpu False
bf16 False
memory_efficient_bf16 False
fp16 True
memory_efficient_fp16 False
fp16_no_flatten_grads False
fp16_init_scale 128
fp16_scale_window None
fp16_scale_tolerance 0.0
on_cpu_convert_precision False
min_loss_scale 0.0001
threshold_loss_scale None
amp False
amp_batch_retries 2
amp_init_scale 128
amp_scale_window None
user_dir None
empty_cache_freq 0
all_gather_list_size 16384
model_parallel_size 1
quantization_config_path None
profile False
reset_logging False
suppress_crashes False
use_plasma_view False
plasma_path /tmp/plasma
------------------

In [2]:
import torch
torch.manual_seed(0)
x = torch.randint(0, 270000, (4, 16)).to(device)
y = model(x)

pad_token = x.new_full((x.size(0),1), 1)
target = torch.cat((x[:,1:], pad_token),1)

out = model.decoder.adaptive_softmax.get_log_prob(y[0], target=target)
idx0 = 0
idx1 = 9
print(out[idx0,idx1][target[idx0,idx1]])


NameError: name 'model' is not defined

torch.Size([4, 16, 267744])


In [15]:
from fairseq.models.transformer_lm import TransformerLanguageModel
fairseq_model = TransformerLanguageModel.from_pretrained('/base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/', 'model.pt')
device = "cuda:0"

fairseq_model.to(device)
fairseq_model.sample('Barack Obama', beam=5)


2024-08-09 18:19:39 | INFO | fairseq.file_utils | loading archive file /base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/
/home/jovyan/fairseq/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for an

ImportError: Please build Cython components with: `pip install --editable .` or `python setup.py build_ext --inplace`

In [8]:
#First download and prepare the WikiText-103 dataset:
!(cd examples/language_model; \
bash prepare-wikitext-103.sh; \
cd ../..)

1296.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


wikitext-103.tar.gz already exists, skipping download


In [9]:
#Next preprocess/binarize the data:
!(TEXT=examples/language_model/wikitext-103; \
fairseq-preprocess \
    --only-source \
    --trainpref $TEXT/wiki.train.tokens \
    --validpref $TEXT/wiki.valid.tokens \
    --testpref $TEXT/wiki.test.tokens \
    --destdir data-bin/wikitext-103 \
    --workers 20)

1330.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2024-08-05 22:51:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-08-05 22:51:41 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/wikitext-103', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', p

In [2]:
#Evaluate
# Evaluate

# Add your code here
!(fairseq-eval-lm data-bin/wikitext-103 \
    --path /base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/model.pt \
    --batch-size 2 \
    --tokens-per-sample 512 \
    --context-window 400)
# | Evaluated 245569 tokens in 56.1s (4379.02 tokens/s)
# | Loss: 3.4164, Perplexity: 30.46

5.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2024-08-05 22:29:35 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-08-05 22:29:37 | INFO | fairseq_cli.eval_lm | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': F

In [4]:
import re
import sys
from fairseq_cli.eval_lm import cli_main

cli_main("fairseq-eval-lm data-bin/wikitext-103 \
    --path /base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/model.pt \
    --batch-size 2 \
    --tokens-per-sample 512 \
    --context-window 400")

TypeError: cli_main() takes 0 positional arguments but 1 was given

In [7]:
import subprocess

# Define your command as a string
# Replace 'arg1', 'arg2', etc. with your actual arguments
command = "fairseq-eval-lm data-bin/wikitext-103 \
    --path /base-vol-2/fairseq/models/adaptive_lm_wiki103.v2/model.pt \
    --batch-size 2 \
    --tokens-per-sample 512 \
    --context-window 400"

# Use subprocess to run the command
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
process.wait()

# Print the output
print(process.stdout.read().decode())

2024-08-05 22:42:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-08-05 22:42:19 | INFO | fairseq_cli.eval_lm | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': F